# Workflow to analyze microbiome data.
- [x] Downstream metagenomics analysis 
- [x] Downstream 16S analysis would be covered in ./scripts/EasyMicrobiome.Rmd

## Load packages

In [2]:
from scripts import separation
import pandas as pd
import os
import numpy as np
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

In [3]:
# check data
p = "./data/cropps_summer_tax/22/"
plist = [] 
for i in ["per_contig.tax.txt","per_gene.tax_and_kofam.txt","per_tax.total_base_count.txt","prodigal.cds.faa.blastp.add_tax"]:
    plist.append(os.path.join(p,i))

df1 = pd.read_csv(plist[0],sep='\t',header=None)
df1.columns = "seq_id,tax".split(",")
df2 = pd.read_csv(plist[1],delimiter='\t',header=None)
df2.columns = "seq_id,ncbi_accession,identity%,align_length,#mismatch,E-value,bitscore,gene_anno,organism,taxid,KO".split(",")
df3 = pd.read_csv(plist[2],delimiter='\t',header=None)
df3.columns = "tax,base_count".split(",")

df4 = df1.merge(df3,"inner")

df5 = pd.read_csv(plist[3],delimiter='\t',header=None)
df5.columns = "seq_id,ncbi_accession,identity%,align_length,#mismatch,E-value,bitscore,gene_anno,organism,taxid".split(",")
df6 = df5["seq_id"].str.split("_",expand=True)
temp = df6.iloc[:,0]+"_"+df6.iloc[:,1]
df6 = pd.concat([temp,df6[2]],axis=1)
df6.columns = ["contig_id","gene_id"]
df5 = pd.concat([df6,df5.iloc[:,1:]],axis=1)
# now we only handle df2 and df4 and df5

In [151]:
## for the KO identification
# db_all = pd.read_csv("db.ko.list.txt",sep='\t',header=None)
# db_all.columns = ["KO","KEGGanno"]
# # load db 
# with open("db.ko.CNP.json","r") as f:
#     db = json.load(f)
# # df2.replace({"KO":db["PolyP"]})["KO"]
# df = df2.merge(db_all,"inner")

In [4]:
# for the keyword identification for functions
funclist = ["nitrate","nitrite","ammonia","nitric","nitrous","nitrogen","polyphos","phosphate"]
def get_funcidx(df,funclist):
    
    idxlist = {}
    for i in range(len(funclist)):
        idx = df["gene_anno"].str.contains(funclist[i])
        idxlist[funclist[i]] = idx
    return idxlist

def get_intersect(idxi,idxj,df):
    dfi = df.loc[idxi,:]
    dfj = df.loc[idxj,:]
    # print(dfi)
    print(dfj)
    sharedcontig = pd.Series(list(set(dfi["contig_id"]).intersection(set(dfj["contig_id"]))))
    return sharedcontig

idxlist = get_funcidx(df5,funclist)
idxi = idxlist["nitrate"]
idxj = idxlist["polyphos"]
get_intersect(idxi,idxj,df5)

          contig_id gene_id ncbi_accession  identity%  align_length  \
126  c_000000000009       1     EER55977.1      100.0           502   

     #mismatch  E-value  bitscore  \
126          0      0.0     977.0   

                                           gene_anno  \
126  exopolyphosphatase [Neisseria flavescens SK114]   

                       organism   taxid  
126  Neisseria flavescens SK114  596320  


/var/folders/yp/5srx62ln4zxcf2c1k3b3kknr0000gn/T/ipykernel_24506/2963722332.py:16: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sharedcontig = pd.Series(list(set(dfi["contig_id"]).intersection(set(dfj["contig_id"]))))


Series([], dtype: float64)

In [25]:
# let's start from looking forgenus Neisseria
idtax = df5["organism"].str.contains("Staphylococcus")
tax = df5.loc[idtax,:]["gene_anno"]

tax_gene_id = tax.str.contains("ammon")

tax_gene = tax.loc[tax_gene_id]
print(tax_gene)
# idxlist = get_funcidx(tax,funclist=["polyp"])
# tax.loc[idxlist["polyp"],]["gene_anno"]


285      MULTISPECIES: ammonium transporter [Staphyloco...
767      MULTISPECIES: threonine ammonia-lyase IlvA [St...
27520    threonine ammonia-lyase domain protein [Staphy...
Name: gene_anno, dtype: object


## Run it

In [6]:
df5["organism"]

0                  Neisseria mucosa
1        Achromobacter xylosoxidans
2              Neisseria flavescens
3            Neisseria meningitidis
4         Neisseria subflava NJ9703
                    ...            
28063        Staphylococcus hominis
28064              Escherichia coli
28065        Staphylococcus hominis
28066        Staphylococcus hominis
28067        Staphylococcus warneri
Name: organism, Length: 28068, dtype: object